In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import cv2
import pickle
from tensorflow.keras.utils import to_categorical

In [2]:
def load_images(path, label):
    images = []
    for filename in os.listdir(path):
        img = cv2.imread(os.path.join(path, filename))
        if img is not None:
            images.append((img, label))
    return images

In [3]:
closed_eye_images = load_images('./dataset_B_Facial_Images/ClosedFace', 1)
open_eye_images = load_images('./dataset_B_Facial_Images/OpenFace', 0)

In [4]:
images = closed_eye_images + open_eye_images

np.random.shuffle(images)

X = np.array([img[0] for img in images])
Y = np.array([img[1] for img in images])

In [5]:
X_train = np.array([cv2.resize(img, (224,224)) for img in X[:int(X.shape[0]*0.8)]])
Y_train = Y[:int(Y.shape[0]*0.8)]

X_test = np.array([cv2.resize(img, (224,224)) for img in X[int(X.shape[0]*0.8):]])
Y_test = Y[int(Y.shape[0]*0.8):]

In [6]:
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [7]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [8]:
Y_train = to_categorical(Y_train, num_classes=2)[:, 0]
Y_test = to_categorical(Y_test, num_classes=2)[:, 0]

In [9]:
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in base_model.layers:
    layer.trainable = False

In [10]:
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(2048, activation='relu'),
    tf.keras.layers.Reshape((1, 2048)),
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(1, 2048)),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(X_train, Y_train, epochs=5)

Epoch 1/5
61/61 [==============================] - 44s 533ms/step - loss: 0.5456 - accuracy: 0.7234
Epoch 2/5
61/61 [==============================] - 32s 528ms/step - loss: 0.3445 - accuracy: 0.8638
Epoch 3/5
61/61 [==============================] - 33s 537ms/step - loss: 0.2870 - accuracy: 0.8896
Epoch 4/5
61/61 [==============================] - 32s 527ms/step - loss: 0.2717 - accuracy: 0.8958
Epoch 5/5
61/61 [==============================] - 33s 536ms/step - loss: 0.2555 - accuracy: 0.8942


In [13]:
results = model.evaluate(X_test, Y_test)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

16/16 [==============================] - 12s 534ms/step - loss: 0.2218 - accuracy: 0.9134
Test Loss: 0.2218250185251236
Test Accuracy: 0.9134020805358887


In [14]:
Y_pred = model.predict(X_test)
Y_pred = np.round(Y_pred)

16/16 [==============================] - 11s 479ms/step


In [15]:
model_dir = "./_model"

localhost_save_option = tf.saved_model.SaveOptions(experimental_io_device="/job:localhost")
model.save(model_dir, options=localhost_save_option)

INFO:tensorflow:Assets written to: ./_model\assets


INFO:tensorflow:Assets written to: ./_model\assets


In [16]:
import tensorflow as tf
model2 = tf.keras.models.load_model(model_dir)

KeyboardInterrupt: 

In [ ]:
score = model2.evaluate(X_test, Y_test)
print("Test score: {0:.2f} %".format(100 * score[1]))

In [ ]:
Y_predict = model2.predict(X_test)  
Y_predict = np.round(Y_predict)
Y_predict